# 1. 문서를 읽는다

In [2]:
%pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 KB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 49.5 MB/s eta 0:00:0031m54.6 MB/s eta 0:00:01
You should consider upgrading via the '/Users/righteous/.pyenv/versions/address-llm-application/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [6]:
from docx import Document

document = Document('./tax.docx')
print(f'document == {dir(document)}')
full_text = ''

for index, paragraph in enumerate(document.paragraphs):
    print(f'paragraph == {paragraph.text}')
    full_text += f'{paragraph.text}\n'

document == ['_Document__body', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_block_width', '_body', '_element', '_parent', '_part', 'add_comment', 'add_heading', 'add_page_break', 'add_paragraph', 'add_picture', 'add_section', 'add_table', 'comments', 'core_properties', 'element', 'inline_shapes', 'iter_inner_content', 'paragraphs', 'part', 'save', 'sections', 'settings', 'styles', 'tables']
paragraph == 소득세법
paragraph == [시행 2025. 10. 1.] [법률 제21065호, 2025. 10. 1., 타법개정]
paragraph == 기획재정부(재산세제과(양도소득세)) 044-215-4312
paragraph == 기획재정부(소득세제과(근로소득)) 044-215-4216
paragraph == 기획재정부(금융세제과(이자소득, 배당소득)) 044-215-4233
paragraph == 기획재정부(소득세제과(사업소득, 기타소득)) 044-215-4217
paragraph == 
paragraph == 제1

# 2. 문서를 쪼갠다

In [8]:
%pip install tiktoken

You should consider upgrading via the '/Users/righteous/.pyenv/versions/address-llm-application/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [12]:
import tiktoken

def split_text(full_text, chunk_size):
    encoder = tiktoken.encoding_for_model("gpt-4o")
    total_encoding = encoder.encode(full_text)
    total_token_count = len(total_encoding)
    text_list = []

    for i in range(0, total_token_count, chunk_size):
        chunk = total_encoding[i: i + chunk_size]
        decoded = encoder.decode(chunk)
        text_list.append(decoded)
    return text_list

In [ ]:
chunk_list = split_text(full_text=full_text, chunk_size=1500)

['소득세법\n[시행 2025. 10. 1.] [법률 제21065호, 2025. 10. 1., 타법개정]\n기획재정부(재산세제과(양도소득세)) 044-215-4312\n기획재정부(소득세제과(근로소득)) 044-215-4216\n기획재정부(금융세제과(이자소득, 배당소득)) 044-215-4233\n기획재정부(소득세제과(사업소득, 기타소득)) 044-215-4217\n\n제1장 총칙 <개정 2009. 12. 31.>\n\n제1조(목적) 이 법은 개인의 소득에 대하여 소득의 성격과 납세자의 부담능력 등에 따라 적정하게 과세함으로써 조세부담의 형평을 도모하고 재정수입의 원활한 조달에 이바지함을 목적으로 한다.\n[본조신설 2009. 12. 31.]\n[종전 제1조는 제2조로 이동 <2009. 12. 31.>]\n\n제1조의2(정의) ① 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2010. 12. 27., 2014. 12. 23., 2018. 12. 31.>\n1. “거주자”란 국내에 주소를 두거나 183일 이상의 거소(居所)를 둔 개인을 말한다.\n2. “비거주자”란 거주자가 아닌 개인을 말한다.\n3. “내국법인”이란 「법인세법」 제2조제1호에 따른 내국법인을 말한다.\n4. “외국법인”이란 「법인세법」 제2조제3호에 따른 외국법인을 말한다.\n5. “사업자”란 사업소득이 있는 거주자를 말한다.\n② 제1항에 따른 주소ㆍ거소와 거주자ㆍ비거주자의 구분은 대통령령으로 정한다.\n[본조신설 2009. 12. 31.]\n\n제2조(납세의무) ① 다음 각 호의 어느 하나에 해당하는 개인은 이 법에 따라 각자의 소득에 대한 소득세를 납부할 의무를 진다.\n1. 거주자\n2. 비거주자로서 국내원천소득(國內源泉所得)이 있는 개인\n② 다음 각 호의 어느 하나에 해당하는 자는 이 법에 따라 원천징수한 소득세를 납부할 의무를 진다.\n1. 거주자\n2. 비거주자\n3. 내국법인\n4. 외국법인의 국내지점 또는 국내영업소(출장소, 그 밖에 이에 준하는 것을 포함한다. 이하 

# 3. 문서 임베딩

In [14]:
%pip install chromadb

You should consider upgrading via the '/Users/righteous/.pyenv/versions/address-llm-application/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [15]:
import chromadb

chroma_client = chromadb.Client()

In [16]:
collection_name = 'tax_collection'
tax_collection = chroma_client.create_collection(collection_name)

In [ ]:
import os
from dotenv import load_dotenv
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
openai_embedding = OpenAIEmbeddingFunction(model_name='text-embedding-3-large', api_key=openai_api_key)


In [ ]:
tax_collection = chroma_client.get_or_create_collection(name=collection_name, embedding_function=openai_embedding)

ValueError: An embedding function already exists in the collection configuration, and a new one is provided. If this is intentional, please embed documents separately. Embedding function conflict: new: openai vs persisted: default

In [ ]:
id_list = []
for index in range(len(chunk_list)):
    id_list.append(f'{index}')